In [ ]:
# 1.值范围 2.相似线段形态 3.highlight落在区间内 4.方差大

In [6]:
a = 1
if a == {}:
    print(True)

In [7]:
import re
str1 = '[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}'
re.findall('\{(.*?)\}', str1)

['timestamp', '', 'txAtt:d', '', 'torTemperature:d', '', 'avgIMpa0:d', '']

In [ ]:
import json
import uuid
branch = ['A','B','C','D','E','F','G','H','I','J','K','L']
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}
for index in range(12):
    item = {}
    item['role_path'] = 'root.' + 'branch' + str(branch[index]) + '.search_branch' + str(branch[index])
    item['identifier'] = 'search_branch' + str(branch[index])
    item['desc'] = 'Branch ' + str(branch[index]) + ' common use KeyValue and Mark'
    item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+').*(txAtt|Pma|linearization fault)'
    item['exp_extract'] = [
                        "{}[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
                        # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
                        "{}[{timestamp}] {}"
                        ]
    item['exp_mark'] = [{"abbr":"LF","exp":"linearization fault","color":"#f00000"}]
    item['is_active'] = False
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    if index == 0:
        item['is_active'] = True
    config['search'].append(item)

keys = ['txAtt', 'torTemperature', 'avgIMpa0', 'LF']
for search_config in config['search']:
    item = {}
    item['role_path'] = search_config['role_path'].replace('search', 'chart')
    item['identifier'] = search_config['identifier'].replace('search', 'chart')
    item['desc'] = item['identifier'] + ' all KeyValue and Mark'
    item['check'] = False
    item['key_value_tree'] = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
    for search_config2 in config['search']:
        search_atom = {}
        search_atom['namespace'] = search_config2['identifier']
        search_atom['name'] = search_config2['identifier']
        search_atom['check'] = False
        search_atom['children'] = []
        for key in keys:
            if search_config['identifier'] == search_config2['identifier']:
                search_atom['children'].append({'name': key, 'check': True})
            else:
                search_atom['children'].append({'name': key, 'check': False})
        item['key_value_tree']['children'].append(search_atom)
    config['chart'].append(item)

# for index in range(12):
#     item = {}
#     item['name'] = 'insight_branch' + str(branch[index])
#     item['desc'] = 'Branch Insight' + str(branch[index])
#     item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+')'
#     item['exp_extract'] = "{}[{timestamp}]{}, msg = {msg}"
#     item['exp_mark'] = {"name":"LF","exp":"linearization fault","color":"#f00000"}
#     item['is_case_sensitive'] = True
#     item['forward_rows'] = 0
#     item['backward_rows'] = 0
#     config['insight'].append(item)

# for index in range(12):
#     item = {}
#     item['name'] = 'statistic_branch' + str(branch[index])
#     item['desc'] = 'Branch ' + str(branch[index]) + ' statistic txAtt'
#     item['code'] = "self.result = self.text_file_model.data['" + 'search_branch' + str(branch[index]) + "'].res_key_value['txAtt']['global_index']"
#     config['statistic'].append(item)
    
json_object = json.dumps(config)
with open("E:\\projects\\ericsson_flow\\new_files\\config.ecfg", "w") as outfile:
    outfile.write(json_object)

In [10]:
import json
import uuid
import re

def add_search_config(parent, express, processes, keyword, mark):
    config = []
    
    item = {}
    item['identifier'] = f'{parent}_ErrorFaultABN'
    identifier = item['identifier']
    item['role_path'] = f'root.{parent}.EFA.{identifier}'

    item['desc'] = parent + ' Error Fault ABN'
    item['exp_search'] = parent + '.*(error| fault|ABN)'
    item['exp_extract'] = [
                            parent + express
                        ]
    item['exp_mark'] = [
                            {"abbr":"ER","exp":"error","color":"#f00000"},
                            {"abbr":"FA","exp":" fault","color":"#f00000"},
                            {"abbr":"ABN","exp":"ABN","color":"#ffff00"}
                        ]
    item['is_active'] = True
    item['is_case_sensitive'] = False
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config.append(item)

    for index, process in enumerate(processes):
        item = {}
        name = process.replace(',','_')
        item['identifier'] = f'{parent}_{name}'
        identifier = item['identifier']
        item['role_path'] = f'root.{parent}.{identifier}' if (keyword not in process) else f'root.{parent}.{process}.{identifier}'
        item['desc'] =  parent + ' ' + process + ' common use KeyValue and Mark'
        item['exp_search'] = f'{parent}.*{process}'
        item['exp_extract'] = [
                                parent + express
                            ]
        item['exp_mark'] = mark if (keyword in process) else []
        item['is_active'] = False
        item['is_case_sensitive'] = True
        item['forward_rows'] = 0
        item['backward_rows'] = 0
        config.append(item)
    return config
        
def add_chart_config(tmp_search):
    config = []
    for search_config in tmp_search:

        if len(search_config['exp_mark']) == 0:
            continue

        item = {}
        item['role_path'] = search_config['role_path'] + '_Chart'
        item['identifier'] = search_config['identifier'] + '_Chart'
        item['desc'] = item['identifier'] + ' all KeyValue and Mark'
        item['check'] = False
        item['key_value_tree'] = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
        for search_config2 in tmp_search:
            search_atom = {}
            search_atom['namespace'] = search_config2['identifier']
            search_atom['name'] = search_config2['identifier']
            search_atom['check'] = False
            search_atom['children'] = []
            if len(search_config2['exp_mark']) > 0:
                keys = []
                for mark in search_config2['exp_mark']:
                    keys.append(mark['abbr'])
                    
                for key in keys:
                    if search_config['identifier'] == search_config2['identifier']:
                        search_atom['children'].append({'name': key, 'check': True})
                    else:
                        search_atom['children'].append({'name': key, 'check': False})
                        
            item['key_value_tree']['children'].append(search_atom)
        config.append(item)
    return config

config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\DOT TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall(' \((.*?)\): ', line)
        if len(res) > 0:
            processes.append(res[0]) 
processes = list(set(processes))
processes.sort()
config['search'].extend(add_search_config('DOT_TE_LOG', ":{timestamp} ({}", processes, 'kkkkkkkkkkkkk', []))

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\DU1 TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall('procname = \"(.*?)\"', line)
        if len(res) > 0:
            processes.append(res[0])
processes = list(set(processes))
processes.sort()
config['search'].extend(add_search_config('DU1_TE_LOG', ":[{timestamp}]{}", processes, 'kkkkkkkkkkkkk', []))
config['search'].extend(add_search_config('DU2_TE_LOG', ":[{timestamp}]{}", processes, 'kkkkkkkkkkkkk', []))

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\RU1 TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall(', { \"(.*?)\"', line)
        if len(res) > 0:
            processes.append(res[0])
processes = list(set(processes))
processes.sort()
mark = [
            {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#759aa0"},
            {"abbr":"ER","exp":"EVENT_RELEASE","color":"#e69d87"},
            {"abbr":"ES","exp":"EVENT_SETUP","color":"#8dc1a9"},
            {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#ea7e53"},
            {"abbr":"IND","exp":"IND_ACTIVATE","color":"#eedd78"}
        ]
tmp_search = add_search_config('RU1_TE_LOG', '{}[{timestamp}]{}', processes, 'TxBranchCtrl', mark)
config['search'].extend(tmp_search)
config['chart'].extend(add_chart_config(tmp_search))

tmp_search = add_search_config('RU2_TE_LOG', '{}[{timestamp}]{}', processes, 'TxBranchCtrl', mark)
config['search'].extend(tmp_search)
config['chart'].extend(add_chart_config(tmp_search))

json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\DotConfigFull.ecfg", "w") as outfile:
    outfile.write(json_object)

In [ ]:
import re
from parse import parse

str1 = 'RU2_TE_LOG:BXP_2: [2023-02-23 10:31:25.766459124] com_ericsson_trithread:INFO: { 0 }, { "TxBranchCtrlC", "logManager.cc:37", "Branch does not support different power level (txChangeCycleStatePrepare.cc:517)" }'
# str2 = 'BXP_3: [2022-12-10 15:55:26.739019220] (+0.000025340) radio6626 com_ericsson_trithread:INFO: { cpu_id = 1 }, { process = "TxBranchCtrlB", fileAndLine = "txChangeCycleHelper.cc:264", msg = "Txl branch J restart due to txL linearization fault!" }'
# str1 = "BXP_2: [221120 164014] 27: PA measured values for driver name: DpaVddSv:7; value: 26992; branch Id: 7"
# exp = "\[%{TIMESTAMP:time}\] \(%{STRING:cost}\) "
# exp = "\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] "
exp_extract = '{}[{timestamp}]{}'
r_extract = parse(exp_extract, str1)
print(r_extract)

In [12]:
import re

# processes = []
# with open("D:\\projects\\ericsson_flow\\new_files\\passlog.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         res = re.findall("process = \"(.*?)\",", line)
#         if len(res) > 0:
#             if ('BranchCtrl' not in res[0]) & ('PhaseCtrl' not in res[0]):
#                 processes.append(res[0])
            
# processes = list(set(processes))
# processes.sort()
processes = [
 'EquipCtrl',
 'PaSrvAB',
 'PimcTdiHandler',
 'RICRAI_SERVER',
 'RxBranchCtrlA',
 'RxBranchCtrlB',
 'RxBranchCtrlC',
 'RxBranchCtrlD',
 'RxRmcService',
 'TorLoSrvAB',
 'TxBranchCtrlA',
 'TxBranchCtrlB',
 'TxCoordinationSrvAB',
 'TxLoSrvA',
 'TxLoSrvB',
 'TxTimingPhaseCtrlB',
 'antpServerProc_',
 'bcProc',
 'brhProc',
 'child.c',
 'cmd_proc',
 'cmd_proc(COLI)',
 'cpriCtrlProc',
 'cprilhmd.c',
 'ecp_x11.c',
 'faultManagerProc',
 'ledProc',
 'libecp_x11.c',
 'radioTrDcServer',
 'rftProc',
 'rhd-lh.c',
 'rmcProc',
 'rtsCtrl',
 'stimenotifyd.c',
 'tmoSchedulerEqp',
 'tmoSchedulerTx',
 'trDcProc',
 'txlProcBranchA',
 'txlProcBranchB',
 'ulh_hdlc.c']

In [22]:
import json
import uuid

product_name = '4449'
processes = processes
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

item = {}
item['role_path'] = f'{product_name}.TimedoutErrorFaultABN'
item['identifier'] = 'TimedoutErrorFaultABN'
item['desc'] = 'Error Fault ABN Timedout'
item['exp_search'] = 'error| fault|ABN|Timed out'
item['exp_extract'] = [
                    "[{timestamp}] {}" 
                    ]
item['exp_mark'] = [
                        {"abbr":"TO","exp":"Timed out","color":"#FFB6C1"},
                        {"abbr":"ER","exp":"error","color":"#f00000"},
                        {"abbr":"LF","exp":"linearization fault","color":"#f00000"},
                        {"abbr":"ABN","exp":"ABN:","color":"#ffff00"}
                    ]
item['is_active'] = True
item['is_case_sensitive'] = True
item['forward_rows'] = 0
item['backward_rows'] = 0
config['search'].append(item)

for index, process in enumerate(processes):
    item = {}
    item['role_path'] = f'{product_name}.' + process.replace('.','_')
    item['identifier'] = process
    item['desc'] =  process + ' common use KeyValue and Mark'
    item['exp_search'] = '"'+process+'", '
    item['exp_extract'] = [
                        # "[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
                        # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
                        "[{timestamp}] {}" 
                        ]
    # item['exp_mark'] = [
    #                         {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#e00000"},
    #                         {"abbr":"ER","exp":"EVENT_RELEASE","color":"#d00000"},
    #                         {"abbr":"ES","exp":"EVENT_SETUP","color":"#c00000"},
    #                         {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#b00000"},
    #                         {"abbr":"IND","exp":"IND_ACTIVATE","color":"#a00000"}
    #                     ]
    item['is_active'] = False
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config['search'].append(item)

# tmp_search = []
# processes = range(2)
# for index in processes:
#     item = {}
#     branch_num = str(index)
#     item['role_path'] = 'root.' + 'branch' + branch_num + '.search_branch' + branch_num
#     item['identifier'] = 'search_branch' + branch_num
#     item['desc'] = 'Branch ' + branch_num + ' common use KeyValue and Mark'
#     item['exp_search'] = f'(process = "txlProcBranch'+branch_num+'"|process = "TxBranchCtrl'+branch_num+'")'
#     item['exp_extract'] = [
#                         "[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
#                         # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
#                         "[{timestamp}] {}" 
#                         ]
#     item['exp_mark'] = [
#                             {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#759aa0"},
#                             {"abbr":"ER","exp":"EVENT_RELEASE","color":"#e69d87"},
#                             {"abbr":"ES","exp":"EVENT_SETUP","color":"#8dc1a9"},
#                             {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#ea7e53"},
#                             {"abbr":"IND","exp":"IND_ACTIVATE","color":"#eedd78"}
#                         ]
#     item['is_active'] = False
#     item['is_case_sensitive'] = True
#     item['forward_rows'] = 0
#     item['backward_rows'] = 0
#     config['search'].append(item)
#     tmp_search.append(item)
        
# keys = ['txAtt', 'avgIMpa0', 'ED', 'ER', 'ES', 'EA', 'IND']
# for search_config in tmp_search:
#     item = {}
#     item['role_path'] = search_config['role_path'].replace('search', 'chart')
#     item['identifier'] = search_config['identifier'].replace('search', 'chart')
#     item['desc'] = item['identifier'] + ' all KeyValue and Mark'
#     item['check'] = False
#     item['key_value_tree'] = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
#     for search_config2 in tmp_search:
#         search_atom = {}
#         search_atom['namespace'] = search_config2['identifier']
#         search_atom['name'] = search_config2['identifier']
#         search_atom['check'] = False
#         search_atom['children'] = []
#         for key in keys:
#             if search_config['identifier'] == search_config2['identifier']:
#                 search_atom['children'].append({'name': key, 'check': True})
#             else:
#                 search_atom['children'].append({'name': key, 'check': False})
#         item['key_value_tree']['children'].append(search_atom)
#     config['chart'].append(item)
    
json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\4449CommonConfig.ecfg", "w") as outfile:
    outfile.write(json_object)

In [21]:
if '' in '123':
    print(True)

True


In [ ]:
import re

text = "I like oranges."
pattern = "^(?!.*jesd|.*dfe|.*radiosw|.*rProxyMedian.cc).*" (txlProcBranch0|TxBranchCtrl0).*(event|IND).*328236

re.findall(pattern, text)

In [ ]:
import json
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

item1 = {}
item1['name'] = 'PackageName'
item1['desc'] = 'Elog package restart name'
item1['exp_search'] = '_R13C193'
item1['exp_extract'] = [
                    "[{tmp1}] {}LMC ID: {packname}\n[{tmp2}] {}\n[{tmp3}] {}\n[{tmp4}] {}\n[{tmp5}] {}\n[{timestamp}] {}"
                    ]
item1['exp_mark'] = []
item1['is_case_sensitive'] = True
item1['forward_rows'] = 0
item1['backward_rows'] = 5
config['search'].append(item1)

item2 = {}
item2['name'] = 'ErrorName'
item2['desc'] = 'ABN: raiseFaultAndRollback'
item2['exp_search'] = 'ABN: raiseFaultAndRollback'
item2['exp_extract'] = [
                    "[{timestamp}] {}ABN: {errorname} {}"
                    ]
item2['exp_mark'] = []
item2['is_case_sensitive'] = True
item2['forward_rows'] = 0
item2['backward_rows'] = 0
config['search'].append(item2)

json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\cooper_config.txt", "w") as outfile:
    outfile.write(json_object)

In [ ]:
import re
from parse import parse

str1 = 'DOT_TE_LOG:2023-02-23 10:32:14:027 (srv,info): clgc errorStatus: 13581 '
# str2 = 'BXP_3: [2022-12-10 15:55:26.739019220] (+0.000025340) radio6626 com_ericsson_trithread:INFO: { cpu_id = 1 }, { process = "TxBranchCtrlB", fileAndLine = "txChangeCycleHelper.cc:264", msg = "Txl branch J restart due to txL linearization fault!" }'
# str1 = "BXP_2: [221120 164014] 27: PA measured values for driver name: DpaVddSv:7; value: 26992; branch Id: 7"
# exp = "\[%{TIMESTAMP:time}\] \(%{STRING:cost}\) "
# exp = "\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] "
exp_extract = 'DOT_TE_LOG:{timestamp} ({}'
r_extract = parse(exp_extract, str1)
print(r_extract)

In [ ]:
from parse import parse
# str1 = [
#     "221211-00:51:48 10.69.81.43 22.0n MSRBS_NODE_MODEL_22.Q2_566.28125.116_3317 stopfile=/tmp/21234\n",
#     "coli>/fruacc/lhsh BXP_5 ts r\n",
#     "Board                     : 45.2 C\n",
#     "DcPaVdd:0                 : 57.3 C\n",
#     "DcPaVdd:1                 : 57.8 C\n",
#     "DcPaVdd:2                 : 50.1 C\n",
#     "DcPaVdd:3                 : 50.6 C\n",
#     "DcTrxVcc                  : 48.3 C\n"
# ]

st1 = ['221211-00:50:33 10.69.81.43 22.0n MSRBS_NODE_MODEL_22.Q2_566.28125.116_3317 stopfile=/tmp/21234\n', 'coli>/fruacc/lhsh BXP_5 ts r\n', 'Board                     : 33.2 C\n', 'DcPaVdd:0                 : 43.7 C\n', 'DcPaVdd:1                 : 44.3 C\n', 'DcPaVdd:2                 : 37.4 C\n', 'DcPaVdd:3                 : 37.7 C\n', 'DcTrxVcc                  : 48.9 C\n']
str2 = '\n'.join(str1)
r = parse("{timestamp} {}DcPaVdd:0                 : {temp:f} C{}", str2)
r

In [ ]:
import re
import json
from text_analysis import TextAnalysisModel

text_analysis_model = await TextAnalysisModel('parallel', mode = 'test')

file_path = ['D:\\projects\\ericsson_flow\\ELOG_READ_FULL_ENM3\\DL_BAYAHBARATCISIIH_GH.log']
config_path = 'D:\\Projects\\ericsson_flow\\new_files\\cooper_config1.txt'

await text_analysis_model.file_container_model.on_new_file('', file_path)
await text_analysis_model.file_container_model.on_load_config('', config_path)

for key in text_analysis_model.data.keys():
    if 'Name' in key:
        print(key, text_analysis_model.data[key].res_key_value)
    

In [ ]:
import re
import json
from text_analysis import TextAnalysisModel

text_analysis_model = await TextAnalysisModel('parallel', mode = 'test')

dir_path = 'D:\\projects\\ericsson_flow\\batch_test'
config = 'D:\\projects\\ericsson_flow\\new_files\\config2.txt'

await text_analysis_model.file_container_model.batch_insight_model.new(dir_path, config)

In [ ]:
text_analysis_model.file_container_model.batch_insight_model.result.loc[1, 'resOutlier']

In [ ]:
text_analysis_model.file_container_model.batch_insight_model.samples

In [ ]:
tmp = text_analysis_model.file_container_model.batch_insight_model.result
res = tmp.loc[(tmp['fileName'] == 'E55H060478_LE_JLADANSUKAHATI_CBX_PL_BXP_6_telog.log'), :].reset_index(drop=True).loc[0, :]
dict(res)